In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split
from docplex.mp.model import Model

# Đọc dữ liệu
df = pd.read_csv('australia.txt', header=None, sep=" ")
X = df.iloc[:, 1:]  # Lấy toàn bộ các cột từ cột thứ 2 đến cuối
y = df.iloc[:, 0]   # Lấy toàn bộ cột đầu tiên

# Chuyển đổi nhãn lớp từ 0,1 sang -1,1 nếu cần
y = np.where(y == 0, -1, 1)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Các tham số của mô hình 
c = [1.0] * X_train.shape[1]
l = [-1] * X_train.shape[1]  # Giới hạn dưới của trọng số wj
u = [1] * X_train.shape[1]   # Giới hạn trên của trọng số wj

# Áp dụng 10-fold cross-validation để tìm giá trị tối ưu cho B
best_B = None
best_mean_cv_accuracy = 0

# Danh sách các giá trị B cần kiểm tra
B_values = [i for i in range(1,14)]

for B in B_values:
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cv_accuracies = []

    for train_index, val_index in kf.split(X_train):
        X_cv_train, X_cv_val = X_train.iloc[train_index], X_train.iloc[val_index] #Lấy dataframe của [train_index] cột
        y_cv_train, y_cv_val = y_train[train_index], y_train[val_index] 

        # Khởi tạo mô hình
        opt_mod = Model(name='MILP-SVM based on l1 norm')

        # Số lượng mẫu và số lượng thuộc tính
        m, n = X_cv_train.shape

        # Các biến quyết định
        w = opt_mod.continuous_var_list(n, name='w')
        b = opt_mod.continuous_var(name='b')
        v = opt_mod.binary_var_list(n, name='v')
        xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

        # Hàm mục tiêu
        opt_mod.minimize(opt_mod.sum(xi[i] for i in range(m)))

        # Ràng buộc
        for i in range(m):
            opt_mod.add_constraint(y_cv_train[i] * (opt_mod.sum(w[j] * X_cv_train.iloc[i, j] for j in range(n)) + b) >= 1 - xi[i])
        for j in range(n):
            opt_mod.add_constraint(w[j] <= v[j] * u[j])
            opt_mod.add_constraint(w[j] >= l[j] * v[j])
        opt_mod.add_constraint(opt_mod.sum(c[j] * v[j] for j in range(n)) <= B)

        # Giải quyết mô hình
        solution = opt_mod.solve()

        if solution:
            w_opt = np.array([solution.get_value(f'w_{i}') for i in range(n)])
            b_opt = solution.get_value('b')

            y_cv_pred = np.sign(np.dot(X_cv_val, w_opt) + b_opt)
            cv_accuracy = accuracy_score(y_cv_val, y_cv_pred)
            cv_accuracies.append(cv_accuracy)

    # Đánh giá trung bình trên tập huấn luyện bằng 10-fold cross-validation
    mean_cv_accuracy = np.mean(cv_accuracies) 
    print(f'Mean CV Accuracy with B={B}: {mean_cv_accuracy * 100:.2f}%')

    # Cập nhật giá trị B tối ưu
    if mean_cv_accuracy > best_mean_cv_accuracy:
        best_B = B
        best_mean_cv_accuracy = mean_cv_accuracy

print(f'Best B value: {best_B}')

# Huấn luyện mô hình cuối cùng trên toàn bộ tập huấn luyện với giá trị B tối ưu
opt_mod = Model(name='MILP-SVM based on l1 norm')
m, n = X_train.shape

w = opt_mod.continuous_var_list(n, name='w')
b = opt_mod.continuous_var(name='b')
v = opt_mod.binary_var_list(n, name='v')
xi = opt_mod.continuous_var_list(m, lb=0, name='xi')

opt_mod.minimize(opt_mod.sum(xi[i] for i in range(m)))

for i in range(m):
    opt_mod.add_constraint(y_train[i] * (opt_mod.sum(w[j] * X_train.iloc[i, j] for j in range(n)) + b) >= 1 - xi[i])
for j in range(n):
    opt_mod.add_constraint(w[j] <= v[j] * u[j])
    opt_mod.add_constraint(w[j] >= l[j] * v[j])
opt_mod.add_constraint(opt_mod.sum(c[j] * v[j] for j in range(n)) <= best_B)

solution = opt_mod.solve()

if solution:
    print(solution)
    print()
    w_opt = np.array([solution.get_value(f'w_{i}') for i in range(n)])
    b_opt = solution.get_value('b')

    y_test_pred = np.sign(np.dot(X_test, w_opt) + b_opt)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    print(f'Accuracy on test set: {test_accuracy * 100:.2f}%')
    selected_features = [j for j in range(n) if v[j].solution_value >= 0.5]
    w_values = solution.get_values(opt_mod.find_matching_vars('w'))
    b_value = solution.get_value('b')
    print(f"w: {w_values}")
    print(f"b: {b_value}")
    print(selected_features)
    print(len(selected_features))
else:
    print("No solution found")

Mean CV Accuracy with B=1: 67.76%
Mean CV Accuracy with B=2: 67.76%
Mean CV Accuracy with B=3: 67.76%
Mean CV Accuracy with B=4: 67.76%
Mean CV Accuracy with B=5: 67.76%
Mean CV Accuracy with B=6: 67.76%
Mean CV Accuracy with B=7: 67.76%
Mean CV Accuracy with B=8: 67.76%
Mean CV Accuracy with B=9: 67.76%
Mean CV Accuracy with B=10: 67.76%
Mean CV Accuracy with B=11: 67.76%
Mean CV Accuracy with B=12: 67.76%
Mean CV Accuracy with B=13: 67.76%
Best B value: 1
solution for: MILP-SVM based on l1 norm
objective: 356
status: OPTIMAL_SOLUTION(2)
b=1.000
v_13=1
xi_2=2.000
xi_5=2.000
xi_14=2.000
xi_15=2.000
xi_19=2.000
xi_20=2.000
xi_25=2.000
xi_26=2.000
xi_31=2.000
xi_34=2.000
xi_38=2.000
xi_39=2.000
xi_49=2.000
xi_51=2.000
xi_53=2.000
xi_56=2.000
xi_58=2.000
xi_62=2.000
xi_65=2.000
xi_67=2.000
xi_70=2.000
xi_72=2.000
xi_75=2.000
xi_76=2.000
xi_82=2.000
xi_85=2.000
xi_86=2.000
xi_89=2.000
xi_92=2.000
xi_96=2.000
xi_97=2.000
xi_107=2.000
xi_109=2.000
xi_110=2.000
xi_112=2.000
xi_113=2.000
xi_11